### Spark Session

Entry point to PySpark's functionality within a program. 

In [2]:
### Building SparkSession

from pyspark.sql import SparkSession   ### SparkSession entry point located in pyspark.sql pkg, providing functionality for data transformation

spark = (SparkSession
         .builder     ### Builder pattern abstraction for constructing a sparksession, where we chain the methods to configure the entry point.
         .appName("Analyzing the vocabulary of Pride and Prejudice.") ### Relevant appName helping in identifying which programs run on the Spark cluster
         .getOrCreate()) ### Program works in both interactive or batch mode by avoiding creating a new session if one already exists.

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/09 18:01:28 WARN Utils: Your hostname, OnePiece, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/09 18:01:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/09 18:01:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext ### For using low-level RDD

""" SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD"""



' SparkSession is the wrapper around sparkContext which uses dataframe as it is more versatile and fast as the main datastructure over lower-level RDD'

### Setting the Log Level

Monitoring PySpark jobs is an important part of developing. PySpark has many levels of logging, from nothing to full description of everything happening in cluster. pyspark shell defaults on WARN, which is a bit chatty.  And, PySpark program defaults to INFO level which is oversharing. 

In [4]:
### Changing the log level to keyword

spark.sparkContext.setLogLevel("KEYWORD")

### Keywords: WARN, ALL, INFO, DEBUG, TRACE, OFF, etc.

IllegalArgumentException: requirement failed: Supplied level KEYWORD did not match one of: ALL,DEBUG,ERROR,WARN,INFO,OFF,FATAL,TRACE

In [ ]:
dir(spark.read)

In [6]:
#### Reading the csv file

book = spark.read.text("./gutenberg.txt")

book

DataFrame[value: string]

In [7]:
## To get the schema of the dataframe

book.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
### For documentation use
print(spark.__doc__)

In [ ]:
# To see the contents of the dataframe

book.show(n=10, truncate = False, vertical = True) # Shows 20 rows and truncates long values.
## n = no.of rows, truncate = default truncates at 20 chars, vertical = displays each record as a small table

In [ ]:
"""
Generally spark is lazily evaluated, but you want eager evaluation, similar to pandas you can change the mode

from pyspark.sql import SparkSession

spark = (SparkSession.builder
                     .config("spark.sql.repl.eagerEval.enabled", "True")
                     .getOrCreate())
"""

In [8]:
### Step-2 Tokenize the words 

#### We'll be splitting the lines of text into arrays or words

from pyspark.sql.functions import split

lines = book.select(split(book.value," ").alias("line")) ## Each record is stored in value. Splitting with space as separator

lines. show(5)

+--------------------+
|                line|
+--------------------+
|[The, Project, Gu...|
|                  []|
|[This, eBook, is,...|
|[almost, no, rest...|
|[re-use, it, unde...|
+--------------------+
only showing top 5 rows


In [ ]:
### Otherways to select a value column from the dataframe

from pyspark.sql.functions import col

book.select(book.value)
book.select(book["value"]) # Use this when the col names contains any special chars
book.select(col["value"]) # No need to mention the dataframe
book.select("value") # Might become problematic in case of transformations

In [10]:
### Transforming columns: Splitting a string into a list of words

from pyspark.sql.functions import col, split

lines = book.select(split(col("value"), " "))

lines.printSchema()

root
 |-- split(value,  , -1): array (nullable = true)
 |    |-- element: string (containsNull = false)



In [ ]:
lines.show(5) ## [] represents array, if its empty it means there are no values present or that row is empty

In [11]:
### Aliasing 
book.select(split(col("value"), " ").alias("line")). printSchema() ###You can use .withColumnRenamed method as well to alias


root
 |-- line: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [13]:
lines = book.select(split(book.value, " "))
lines = lines.withColumnRenamed("split(value, , -1)", "line")

In [14]:
lines = book.select(split(book.value, " ").alias("line"))

In [15]:
#### Reshaping the data: Exploding a list into Rows

"""After splitting the records, we have arrays of strings and it would be better to have one record for each word"""

from pyspark.sql.functions import explode, col

words = lines.select(explode(col("line")).alias("word"))

words.show(15)
                                                

+----------+
|      word|
+----------+
|       The|
|   Project|
| Gutenberg|
|     EBook|
|        of|
|     Pride|
|       and|
|Prejudice,|
|        by|
|      Jane|
|    Austen|
|          |
|      This|
|     eBook|
|        is|
+----------+
only showing top 15 rows


In [16]:
### Step-3 Removing punctuation and turning into lower case

from pyspark.sql.functions import lower
words_lower = words.select(lower(col("word")).alias("word_lower"))

words_lower.show()

[Stage 2:>                                                          (0 + 1) / 1]

+----------+
|word_lower|
+----------+
|       the|
|   project|
| gutenberg|
|     ebook|
|        of|
|     pride|
|       and|
|prejudice,|
|        by|
|      jane|
|    austen|
|          |
|      this|
|     ebook|
|        is|
|       for|
|       the|
|       use|
|        of|
|    anyone|
+----------+
only showing top 20 rows


In [17]:
from pyspark.sql.functions import regexp_extract

words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]+" , 0).alias("word")) # We only match for multiple lowercase chars, + will match for one or more occurences

words_clean.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|         |
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
+---------+
only showing top 20 rows


In [18]:
### Filtering Rows

words_nonnull = words_clean.filter(col("word") != "")
words_nonnull.show()

+---------+
|     word|
+---------+
|      the|
|  project|
|gutenberg|
|    ebook|
|       of|
|    pride|
|      and|
|prejudice|
|       by|
|     jane|
|   austen|
|     this|
|    ebook|
|       is|
|      for|
|      the|
|      use|
|       of|
|   anyone|
| anywhere|
+---------+
only showing top 20 rows


In [20]:
### Counting the word frequencies

groups = words_nonnull.groupby(col("word"))

print(groups)

GroupedData[grouping expressions: [word], value: [word: string], type: GroupBy]


In [21]:
results = words_nonnull.groupby(col("word")).count()

results.show()

+-------------+-----+
|         word|count|
+-------------+-----+
|       online|    4|
|         some|  209|
|        still|   72|
|          few|   72|
|         hope|  122|
|        those|   60|
|     cautious|    4|
|    imitation|    1|
|          art|    3|
|      solaced|    1|
|       poetry|    2|
|    arguments|    5|
| premeditated|    1|
|      elevate|    1|
|       doubts|    2|
|    destitute|    1|
|    solemnity|    5|
|   lieutenant|    1|
|gratification|    1|
|    connected|   14|
+-------------+-----+
only showing top 20 rows


In [23]:
from pyspark.sql.functions import col, length

no_of_words_per_lc = words_nonnull.select(length(col("word")).alias("length")).groupby("length").count()

no_of_words_per_lc.show()


+------+-----+
|length|count|
+------+-----+
|    12|  812|
|     1| 4116|
|    13|  393|
|     6| 9276|
|    16|    5|
|     3|28831|
|     5|11998|
|    15|   32|
|     9| 5165|
|    17|    3|
|     4|22213|
|     8| 5121|
|     7| 8679|
|    10| 2455|
|    11| 1386|
|    14|  107|
|     2|23856|
+------+-----+



In [24]:
### Ordering the results on screen

results.orderBy("count", ascending = False).show(10)


+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
|   i| 2052|
|   a| 1997|
|  in| 1920|
| was| 1844|
| she| 1703|
+----+-----+
only showing top 10 rows


In [25]:
### Or, you can order them in this way

results.orderBy(col("count").desc()).show(10)

+----+-----+
|word|count|
+----+-----+
| the| 4496|
|  to| 4235|
|  of| 3719|
| and| 3602|
| her| 2223|
|   i| 2052|
|   a| 1997|
|  in| 1920|
| was| 1844|
| she| 1703|
+----+-----+
only showing top 10 rows


In [26]:
### Writing the df to a file

results.write.csv("./simple_count.csv")


In [ ]:
### To manipulate the partitions use coalesce

results.coalesce(1).write.csv("/count_csv.csv")